### ספריות


In [1]:
import os
import pandas as pd
# from adding_an_addition_following_the_index import adding_an_addition
# from functions import change_Muni_Heb_to_Muni_Eng
# from division_into_traffic_zones_of_plans import division_into_traffic_zones
# from export_geo_layer_for_client_control import export_geo_layer
# from export_index_layer_for_client_control import export_index_layer
# from forecast import clientTaz
# from geographical_features import add_geographical_Features
# from index_layer import index_layer_fun
# from status_exists_for_control import export_status_exists
# from uploading_index_table_elements import uploading_index_table
# from uploading_index_table_elements import uploading_index_table
# from export_forecast_2040 import export_forecast
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

### העלת משתנים להרצת הקוד


In [3]:
cwd = os.getcwd()

create_forecast_basic_ad_hoc_location = r'{}\create_forecast_basic_ad_hoc'.format(cwd)

df_inputs_outputs = pd.read_excel(r'inputs_outputs.xlsx')

### פונקציות


In [4]:
from global_functions import make_point, up_load_gdb, up_load_shp, up_load_df, find_files_with_pattern, make_point, up_load_gdb, up_load_shp

In [5]:
# מיקום לקוח
client_data_folder_location = df_inputs_outputs['location'][0]

# שם תרחיש (forecast_version)
forecast_version = df_inputs_outputs['location'][1]

# v_date
v_date = df_inputs_outputs['location'][2]

# מיקום פלט תחזית בסיס לפי גירסא
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]

# מספר גירסא
forecast_version_number = df_inputs_outputs['location'][1]

index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)

### העלת אזורי תנועה לחישוב


In [6]:
# נתיב תיקייה
folder_path=r'{}\For_approval\Reference_tabels\shp'.format(client_data_folder_location)
gpd_name='tochnit_check.gdb'

forecast = up_load_gdb(r'{}\{}'.format(folder_path,gpd_name),'TAZ_211028_V3_Published_with_client_changes')

### הוספת מאפיינים גיאוגרפים לאזורי תנועה


In [7]:
forecast_point = make_point(forecast)

# Load data layers
DISTRICT = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_PUMA')
jerusalem_city = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_In_jerusal')
subdistrict_il = up_load_gdb(
    r'{}\background_files\subdistrict2008.gdb'.format(cwd), 'subdistrict2008_ITM')
muni_JTMT = up_load_gdb(
    r'{}\background_files\MUNI_border.gdb'.format(cwd), 'muni_under_JTMT_ITM')
    
# מחפש קבצים של jeru_metro_jtmt_border
jeru_metro_jtmt_border_files = find_files_with_pattern(r'{}\background_files'.format(cwd), 'jeru_metro_jtmt_border')
filepath = ''

# לוקח את הקובץ של השכבה
for string in jeru_metro_jtmt_border_files:
    if string.endswith('.shp'):
        filepath = string

# מעלה את השכבה
jeru_metro_jtmt_border = up_load_shp(filepath)

# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
    DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
    urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
    SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
    PUMA)[['Taz_num', 'puma2040_csv_PUMA']]
forecast_point_jerusalem_city = forecast_point.sjoin(
    jerusalem_city)[['Taz_num', 'jerusalem_city']]
forecast_point_subdistrict_il = forecast_point.sjoin(
    subdistrict_il[['geometry', 'ENG_NAME_nafa']])[['Taz_num', 'ENG_NAME_nafa']]
forecast_point_muni_JTMT = forecast_point.query('main_sector!="Palestinian"').sjoin(
    muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num', 'Muni_Heb', 'Sug_Muni', 'CR_PNIM']]
forecast_point_jeru_metro_jtmt_border = forecast_point.sjoin(
    jeru_metro_jtmt_border)[['Taz_num', 'jeru_metro']]

# Merge tables into one table
forecast = (forecast
            .merge(forecast_point_subdistrict_il, on='Taz_num', how='left')
            .merge(forecast_point_muni_JTMT, on='Taz_num', how='left')
            .merge(forecast_point_jeru_metro_jtmt_border, on='Taz_num', how='left')
            .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
            .merge(forecast_point_urban, on='Taz_num', how='left')
            .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
            .merge(forecast_point_PUMA, on='Taz_num', how='left')
            .merge(forecast_point_jerusalem_city, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
forecast.rename(columns={'ENG_NAME_nafa': 'zonetype'}, inplace=True)
forecast.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
forecast.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
forecast.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
forecast.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)


# Data processing for feature columns
forecast.loc[forecast['main_sector'] == 'Palestinian', 'zonetype'] = 'Palestinian'
forecast['in_jerusalem_metropolin'] = 1
forecast.loc[forecast['jeru_metro'] == 0, 'in_jerusalem_metropolin'] = 0
forecast['yosh'] = 0
forecast.loc[forecast['zonetype'] == 'Judea and Samaria', 'yosh'] = 1

# Add constant columns
forecast['REGION'] = 1
forecast['slop'] = 0

forecast = forecast.set_index('Taz_num')
forecast['Taz_num'] = forecast.index

### שינוי עמודה Muni_Heb


In [11]:
muni_english = pd.read_excel(r'{}\background_files\english_names.xlsx'.format(cwd))
muni_heb_mapping = muni_english.set_index('Muni_Heb')

forecast = forecast.merge(muni_heb_mapping, how='left', left_on='Muni_Heb', right_index=True)
forecast.dropna(subset=['Muni_Heb'], inplace=True)

In [8]:
forecast.head()

,Name_hebre,main_sector,classification_name,Shape_Length,Shape_Area,Student_by_Classrooms,geometry,zonetype,Muni_Heb,Sug_Muni,CR_PNIM,jeru_metro,DISTRICT,urban,SCHOOLDISTRICT,PUMA,jerusalem_city,in_jerusalem_metropolin,yosh,REGION,slop,Taz_num
Taz_num,,,,,,,,,,,,,,,,,,,,,,
5410.0,תחנת הקמח,Jewish,כללי צעיר,1108.870355,67684.251395,0,"MULTIPOLYGON Z (((218345.410 632663.297 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,2,2,1,15,1.0,1,0,1,0,5410.0
5402.0,מאפיית אנג'ל,Jewish,כללי צעיר,997.541634,48913.942411,0,"MULTIPOLYGON Z (((218345.410 632663.297 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,2,2,1,15,1.0,1,0,1,0,5402.0
5403.0,גבעת שאול - בית המשפט,Jewish,כללי צעיר,1632.745393,172300.597706,0,"MULTIPOLYGON Z (((217894.273 632767.537 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,999,2,1,999,1.0,1,0,1,0,5403.0
5401.0,גבעת שאול - משרד החינוך,Jewish,כללי צעיר,2413.191203,192278.913000,0,"MULTIPOLYGON Z (((217416.209 633110.278 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,2,2,1,15,1.0,1,0,1,0,5401.0
5405.0,גבעת שאול - למס,Jewish,כללי צעיר,1946.345087,140020.748592,0,"MULTIPOLYGON Z (((217381.552 633308.806 0.000,...",Jerusalem,ירושלים,עירייה,3000,1,999,2,1,999,1.0,1,0,1,0,5405.0
